In [153]:
import pandas
import numpy as np
import datetime
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler

start = datetime.datetime.now()

data = pandas.read_csv('features.csv')

y = data['radiant_win']
X = data.drop([
    'start_time',
    'radiant_win',
    'duration',
    'tower_status_radiant',
    'tower_status_dire',
    'barracks_status_radiant',
    'barracks_status_dire'], axis=1)
    
n_estimators = 30
    
X = X.fillna(0)
X = np.array(X)
y = np.array(y)

sc = StandardScaler()
#X_train_scaled = sc.fit_transform(X)
    
kf = KFold(n_splits=5, shuffle=True)
clf = GradientBoostingClassifier(n_estimators=n_estimators)
print(np.mean(cross_val_score(clf, X, y, cv=kf, scoring='roc_auc')))
        
print('Average time: ', datetime.datetime.now() - start)


0.69004148235
Average time:  0:01:27.911603


Точности при количесве деревьев 10, 20, 30, 40, 50, 100 : 
    0.663568394879
    0.682672314973
    0.689832076468
    0.694402695613
    0.697155505421
    0.706305863469
    Average time:  0:16:24.743392

In [149]:
X_test = np.array(pandas.read_csv('features_test.csv').drop(['start_time'], axis=1).fillna(0))
clf.fit(X, y)
pred = clf.predict_proba(X_test)
#print(X_scaled.shape, X_test.shape)

In [150]:
print(pred[:30])

[[ 0.51113857  0.48886143]
 [ 0.41582562  0.58417438]
 [ 0.52601     0.47399   ]
 [ 0.32209347  0.67790653]
 [ 0.64376052  0.35623948]
 [ 0.4448283   0.5551717 ]
 [ 0.45130461  0.54869539]
 [ 0.40558952  0.59441048]
 [ 0.67883451  0.32116549]
 [ 0.4754733   0.5245267 ]
 [ 0.46760226  0.53239774]
 [ 0.35672798  0.64327202]
 [ 0.65226391  0.34773609]
 [ 0.42559617  0.57440383]
 [ 0.31034296  0.68965704]
 [ 0.57823788  0.42176212]
 [ 0.62887584  0.37112416]
 [ 0.46982781  0.53017219]
 [ 0.47753114  0.52246886]
 [ 0.49822171  0.50177829]
 [ 0.35620379  0.64379621]
 [ 0.32072904  0.67927096]
 [ 0.40014056  0.59985944]
 [ 0.33969639  0.66030361]
 [ 0.33104482  0.66895518]
 [ 0.43060513  0.56939487]
 [ 0.38166003  0.61833997]
 [ 0.60598685  0.39401315]
 [ 0.72273404  0.27726596]
 [ 0.32556748  0.67443252]]


In [144]:
X_train_new = data.drop([
    'start_time',
    'radiant_win',   
    'duration',
    'tower_status_radiant',
    'tower_status_dire',
    'barracks_status_radiant',
    'barracks_status_dire'], axis=1)

X_train_new = X_train_new.fillna(0)

N_heroes = 113
X_pick = np.zeros((X_train_new.shape[0], N_heroes))


for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, X_train_new.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X_train_new.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

for t in range(5):
    X_train_new = X_train_new.drop([
        'r%d_hero' % (t + 1),
        'd%d_hero' % (t + 1)
    ], axis = 1)
X_train_new = np.array(X_train_new)
print(X_train_new.shape)

(97230, 92)


In [145]:
X_train_new = np.hstack((X_train_new, X_pick))
#X_train_new = sc.fit_transform(X_train_new)

In [146]:
from sklearn.linear_model import LogisticRegression

max_acc = 0
good_c = 2.2
C = 1
"""
while C <= 10:
    clf_log = LogisticRegression(C=C)
    res = np.mean(cross_val_score(clf_log, X_train_new, y, cv=kf, scoring='roc_auc'))
    if res > max_acc:
        max_acc = res
        good_c = C
    C += 0.2
"""

"\nwhile C <= 10:\n    clf = LogisticRegression(C=C)\n    res = np.mean(cross_val_score(clf, X_train_new, y, cv=kf, scoring='roc_auc'))\n    if res > max_acc:\n        max_acc = res\n        good_c = C\n    C += 0.2\n"

Наилушая точность достигается при C = 2.2

In [147]:
start = datetime.datetime.now()

#print(np.mean(cross_val_score(clf, X_train_new, y, cv=kf, scoring='roc_auc')))
clf_log = LogisticRegression(C=good_c)
clf_log.fit(X_train_new, y)

X_test_new = pandas.read_csv('features_test.csv').drop(['start_time'], axis=1).fillna(0)

X_pick_log = np.zeros((X_test_new.shape[0], N_heroes))

for i, match_id in enumerate(X_test_new.index):
    for p in range(5):
        X_pick_log[i, X_test_new.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_log[i, X_test_new.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

for t in range(5):
    X_test_new = X_test_new.drop([
        'r%d_hero' % (t + 1),
        'd%d_hero' % (t + 1)
    ], axis = 1)


X_test_new_res = np.hstack((X_test_new, X_pick_log))
print(X_test_new_res.shape)    
pred_log = clf_log.predict_proba(X_test_new_res)

print('Average time: ', datetime.datetime.now() - start)

(17177, 205)
Average time:  0:00:45.073735


In [157]:
#print(np.mean(cross_val_score(clf_log, X_train_new, y, cv=kf, scoring='roc_auc')))
X_train_new_1 = data.drop([
    'start_time',
    'radiant_win',   
    'duration',
    'tower_status_radiant',
    'tower_status_dire',
    'barracks_status_radiant',
    'barracks_status_dire'], axis=1).fillna(0)

for t in range(5):
    X_train_new1 = X_train_new_1.drop([
        'r%d_hero' % (t + 1),
        'd%d_hero' % (t + 1)
    ], axis = 1)

clf_log_1 = LogisticRegression(C=good_c)
print(np.mean(cross_val_score(clf_log_1, np.array(X_train_new_1), y, cv=kf, scoring='roc_auc')))

0.716810413633


In [160]:
min_pred_val = 1
max_pred_val = 0

for t in pred_log:
    min_pred_val = min(min_pred_val, t[1])
    max_pred_val = max(max_pred_val, t[1])
print('Minimum probability: ',  min_pred_val, ' Maximum pribability: ', max_pred_val)

Minimum probability:  0.00475757771297  Maximum pribability:  0.994479007933
